In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
import requests
from bs4 import BeautifulSoup
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
mirae = pd.read_csv('/content/drive/MyDrive/미래에셋/mirae_stockname.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,5,23,34,35,36,40,41,42,46,47,48,62,63,81,92,93,94,98,99,100,104,105,106,120,121,139,150,151,152,156,157,158,162,163,164,178,179,197,208,209,210,214,215,216,220,221,222,236,237,255,266,267,268,272,273,274,278,279,280,294,295,313,324,325,326,330,331,332,336,337,338,352,353,371,382,383,384,388,389,390,394,395,396,410,411,429,440,441,442,446,447,448,452,453,454,468,469,487,498,499,500,504,505,506,510,511,512,526,527,545,556,557,558,562,563,564,568,569,570,584,585,603,614,615,616,620,621,622,626,627,628,642,643,661,672,673,674,678,679,680,684,685,686) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
list(mirae.columns)

['Unnamed: 0',
 '성별',
 '연령',
 '최초계좌개설일',
 '고객등급_202105',
 '주거래상품_202105',
 '주거래매체_202105',
 '국내주식자산_202105',
 '해외주식자산_202105',
 '상품자산_202105',
 '보유종목수_202105',
 '국내주식매수_202105',
 '국내주식매도_202105',
 '해외주식매수_202105',
 '해외주식매도_202105',
 '거래종목수_202105',
 'm.Stock_접속일수_202105',
 'm.Global_접속일수_202105',
 'm.All_접속일수_202105',
 '퇴직연금자산_202105',
 '개인연금자산_202105',
 '코스피거래비중_202105',
 '코스닥거래비중_202105',
 '선호시장_202105',
 '손절횟수_202105',
 '이익실현횟수_202105',
 'DayTrading비중_202105',
 'Swing비중_202105',
 'Buy&Hold비중_202105',
 '시총1천억미만선호도_202105',
 '시총1천억이상3천억미만선호도_202105',
 '시총3천억이상1조미만선호도_202105',
 '시총1조이상선호도_202105',
 '신용_202105',
 '주식보유1위종목_202105',
 '주식보유2위종목_202105',
 '주식보유3위종목_202105',
 '주식보유1위금액_202105',
 '주식보유2위금액_202105',
 '주식보유3위금액_202105',
 '주식매수1위종목_202105',
 '주식매수2위종목_202105',
 '주식매수3위종목_202105',
 '주식매수1위금액_202105',
 '주식매수2위금액_202105',
 '주식매수3위금액_202105',
 '주식매도1위종목_202105',
 '주식매도2위종목_202105',
 '주식매도3위종목_202105',
 '주식매도1위금액_202105',
 '주식매도2위금액_202105',
 '주식매도3위금액_202105',
 'Dr.Big의국내주식_202105'

In [ ]:
#매달 '국내 주식 매도'와 '국내 주식 매수' 칼럼을 모아놓음
maesu_col = []
maedo_col = []
for x in list(mirae.columns):
  if '국내주식매수' in x:
    maesu_col.append(x)
  elif '국내주식매도' in x:
    maedo_col.append(x)

### 수익률 계산 방법

1년동안의 수익률을 계산한 방법은 다음과 같다.

- 총 매수 금액 : 초기에 가지고 있던 자산(국내주식자산_202006) + 1년동안 매수한 모든 금액

- 총 매도 금액 : 1년동안 매도한 모든 금액 + 최종적으로 가지고 있는 자산(국내주식자산_202105)

- 수익률 : (총 매도 금액 - 총 매수 금액) / 총 매수 금액

In [ ]:
mirae['총매수금액'] = mirae['국내주식자산_202006']
mirae['총매도금액'] = mirae['국내주식자산_202105']

#총 매수금액, 총 매도금액을 계산
for i in maesu_col[:-1]:
  mirae['총매수금액'] = mirae['총매수금액'] + mirae[i]
for i in maedo_col[1:]:
  mirae['총매도금액'] = mirae['총매도금액'] + mirae[i]

#수익률 계산
mirae['수익률'] = (mirae['총매도금액'] - mirae['총매수금액']) / mirae['총매수금액']

#수익률이 너무 크게 계산되거나(수익률 50% 이상) 수익률이 너무 작게 계산되면 제외(수익률 -50% 이하)
def overone(x):
  if float(x) >= 0.5 or float(x) <= -0.5:
    return None
  return float(x) * 100
mirae['수익률'] = mirae['수익률'].apply(overone)

del mirae['총매도금액']
del mirae['총매수금액']

In [ ]:
# mirae['수익률'] = mirae['수익률'] * 100

In [ ]:
mirae.shape

(1499992, 701)

In [ ]:
mirae[mirae['수익률'] < 0]['수익률'].value_counts().sum()

209044

In [ ]:
mirae.head()

,Unnamed: 0,성별,연령,최초계좌개설일,고객등급_202105,주거래상품_202105,주거래매체_202105,국내주식자산_202105,해외주식자산_202105,상품자산_202105,보유종목수_202105,국내주식매수_202105,국내주식매도_202105,해외주식매수_202105,해외주식매도_202105,거래종목수_202105,m.Stock_접속일수_202105,m.Global_접속일수_202105,m.All_접속일수_202105,퇴직연금자산_202105,개인연금자산_202105,코스피거래비중_202105,코스닥거래비중_202105,선호시장_202105,손절횟수_202105,이익실현횟수_202105,DayTrading비중_202105,Swing비중_202105,Buy&Hold비중_202105,시총1천억미만선호도_202105,시총1천억이상3천억미만선호도_202105,시총3천억이상1조미만선호도_202105,시총1조이상선호도_202105,신용_202105,주식보유1위종목_202105,주식보유2위종목_202105,주식보유3위종목_202105,주식보유1위금액_202105,주식보유2위금액_202105,주식보유3위금액_202105,...,선호시장_202006,손절횟수_202006,이익실현횟수_202006,DayTrading비중_202006,Swing비중_202006,Buy&Hold비중_202006,시총1천억미만선호도_202006,시총1천억이상3천억미만선호도_202006,시총3천억이상1조미만선호도_202006,시총1조이상선호도_202006,신용_202006,주식보유1위종목_202006,주식보유2위종목_202006,주식보유3위종목_202006,주식보유1위금액_202006,주식보유2위금액_202006,주식보유3위금액_202006,주식매수1위종목_202006,주식매수2위종목_202006,주식매수3위종목_202006,주식매수1위금액_202006,주식매수2위금액_202006,주식매수3위금액_202006,주식매도1위종목_202006,주식매도2위종목_202006,주식매도3위종목_202006,주식매도1위금액_202006,주식매도2위금액_202006,주식매도3위금액_202006,Dr.Big의국내주식_202006,Dr.Big의해외주식_202006,Dr.Big의국내ETF_202006,Dr.Big의펀드연금_202006,Dr.Big의투자진단_202006,Dr.Big의시그널_202006,뉴스로종목포착_202006,해외ETF트렌드_202006,초고수의선택_202006,주간상품TOP10_202006,수익률
0,0,1.0,20대이하,200604.0,A,펀드,2,81000000.0,0.000000e+00,0.000000e+00,0.0,0.0,81000000.0,0.0,0.0,0.0,9.0,0.0,4.0,0.0,0.000000e+00,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27000000.0,NaN,NaN,NaN,38000000.0,26000000.0,11000000.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.907285
1,1,2.0,20대이하,200610.0,A,주식,2,18000000.0,0.000000e+00,0.000000e+00,6.0,9000000.0,8500000.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.000000e+00,95.0,5.0,코스피,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,한진중공업,SK바이오사이언스,한국전력공사,8000000.0,2600000.0,1500000.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.269569
2,2,1.0,60대이상,200001.0,D,주식,2,39000000.0,0.000000e+00,0.000000e+00,1.0,2300000.0,0.0,0.0,0.0,2.0,21.0,0.0,0.0,0.0,0.000000e+00,82.0,18.0,코스피,33.0,38.0,1.0,97.0,0.0,0.0,4.0,13.0,82.0,0.0,효성첨단소재,효성첨단소재,LG전자,13000000.0,12000000.0,5600000.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.799031
3,3,1.0,60대이상,200001.0,S,주식,1,150000000.0,0.000000e+00,2.700000e+09,17.0,130000000.0,100000000.0,0.0,0.0,23.0,29.0,9.0,0.0,0.0,0.000000e+00,73.0,27.0,코스피,27.0,39.0,3.0,94.0,1.0,1.0,8.0,26.0,66.0,4400000.0,HMM,현대제철,삼성전자,18000000.0,15000000.0,12000000.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.722022
4,4,1.0,60대이상,200001.0,V,주식,1,290000000.0,2.500000e+09,9.900000e+10,11.0,15000000.0,6600000.0,7100000.0,0.0,0.0,4.0,1.0,0.0,0.0,8.700000e+10,95.0,5.0,코스피,0.0,0.0,0.0,85.0,14.0,0.0,0.0,0.0,100.0,0.0,삼성전자,삼성SDI,NaN,55000000.0,54000000.0,35000000.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,35000000.0,33000000.0,23000000.0,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,710000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
